# Trabalho 1 de RP

## Importação de bibliotecas

In [1]:
import os
import random

import numpy              as np
import pandas             as pd
import seaborn            as sns
import scipy.stats        as ss
import matplotlib.pyplot  as plt

from collections                import Counter
from sklearn.preprocessing      import MinMaxScaler
from sklearn.model_selection    import train_test_split

## Carregamento dos dados

In [5]:
#caminho absoluto para os dados
audio_path = os.path.realpath('data/audio.txt')
ecg_path = os.path.realpath('data/ecg.txt')

audio = pd.read_fwf(audio_path, header=None).T
ecg = pd.read_fwf(ecg_path, header=None).T

In [6]:
#criando uma nova coluna em cada dataset para definir qual é a classe
#vamos codificar as classes como ecg = 1 e audio = 2
ecg[500] = 1
audio[500] = 2

#juntando os dois conjuntos de dados
dados = pd.concat([audio, ecg]).reset_index(drop=True)
dados[500] = dados[500].astype('float64')
dados.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,-0.005149,-0.001660,-0.003860,-0.003754,0.002709,-0.000123,-0.002560,0.001240,-0.003455,-0.005611,...,0.016021,0.011144,0.001415,-0.002953,-0.012756,-0.016865,-0.015602,-0.018967,-0.014244,2.0
1,-0.117753,-0.191172,-0.095776,-0.069015,-0.298852,-0.224581,-0.088905,-0.148663,0.127519,-0.154734,...,0.766644,0.203609,0.174668,-0.140126,-0.238385,-0.671537,-0.816351,-0.821680,-0.735269,2.0
2,-0.531884,-0.102584,-0.307340,0.154307,-0.455531,-0.031878,-0.081085,-0.378268,-0.187602,0.072513,...,1.363380,0.716238,0.177158,-0.825243,-1.040234,-1.377452,-1.605749,-2.118144,-1.814519,2.0
3,-0.135808,-0.096242,-0.119334,-0.021121,-0.123060,-0.139165,0.019574,-0.125935,0.027199,-0.067951,...,0.314915,0.133274,-0.061850,-0.071172,-0.165596,-0.366608,-0.637880,-0.528150,-0.454956,2.0
4,-0.030297,-0.074934,-0.028709,-0.047723,-0.001417,0.003577,-0.068440,-0.058604,-0.021373,0.022441,...,0.220040,0.175154,0.100520,-0.084716,-0.123486,-0.225727,-0.255724,-0.229396,-0.244727,2.0


## Questão 1 - Criar vetor de atributos dos dados

In [4]:
#calculando as estatísticas dos dados
a1 = [dados.iloc[i, :].mean() for i in range(len(dados))]
a2 = [dados.iloc[i, :].std() for i in range(len(dados))]
a3 = [ss.skew(dados.iloc[i, :]) for i in range(len(dados))]
a4 = [ss.kurtosis(dados.iloc[i, :]) for i in range(len(dados))]
a5 = dados.apply(lambda x: ss.iqr(x), axis=1)

atributos = pd.DataFrame([a1, a2, a3, a4, a5]).T
atributos.columns = ['Média', 'Desvio padrão', 'Assimetria', 'Curtose', 'Intervalo inter-quartil']
atributos.head()

,Média,Desvio padrão,Assimetria,Curtose,Intervalo inter-quartil
0,0.006098,0.090221,21.605849,475.056090,0.020396
1,0.091002,0.554305,-0.258629,-0.789312,0.860058
2,0.158652,1.078906,-0.340964,-0.964726,1.675028
3,0.047457,0.317557,0.113890,1.611331,0.475802
4,0.030319,0.188875,1.993624,21.961859,0.256980


## Questão 2 - Implementação dos algoritmos KNN (K Nearest Neighbors) e MDC (Minimum Distance to Centroid)

In [5]:
class KNN:
  def __init__(self, k):
    self.k = k

  def fit(self, X, y):
    self.X_train = X
    self.y_train = y

  def euclidean_distance(self, vetor, amostra):
    r = list()

    #calcula a distância euclidiana entre a amostra e cada linha do vetor
    for row in vetor:
      r.append(np.sqrt(np.sum(np.square(row - amostra))))
    return r
  
  def calculate_k_neighbors_index(self, X_test):
    distancias = list()

    #calcula a distância euclidiana para cada amostra
    for row in X_test:
      distancias.append(self.euclidean_distance(self.X_train, row))

    #reserva os índices dos elementos ordenados pela distância, da menor para a maior
    indices = np.argsort(distancias)

    #retorna apenas os índices do k vizinhos mais próximos p/ cada amostra (menores distâncias)
    return indices[:, 0:self.k]

  def predict(self, X_test):
    lista_classes = list()
    respostas = list()

    indices_k_vizinhos = self.calculate_k_neighbors_index(X_test)

    #reserva os valores das classes do k vizinhos mais próximos de cada amostra
    for row in indices_k_vizinhos:
      classes_k_vizinhos = self.y_train[row]
      lista_classes.append(classes_k_vizinhos)

    #calcula quantas vezes cada classe aparece entre os k vizinhos, para cada amostra, ordenadas da mais frequente para a menos frequente 
    #o primeiro elemento da tupla, c[0][0], representa a classe mais frequente, ou seja, a resposta do KNN.
    for item in lista_classes:
      c = Counter(item).most_common()
      respostas.append(c[0][0])

    return respostas

  def score(self, X_test, y_real):
    y_predito = self.predict(X_test)

    acertos = 0

    #calcula a proporção de quantas vezes o algoritmo fez uma previsão correta
    for i in range(len(y_real)):
      if y_real[i] == y_predito[i]:
        acertos += 1

    taxa_acerto = acertos / len(y_real)

    return taxa_acerto

### Testando o KNN

In [6]:
X = np.array(dados.drop(500, axis=1))
y = np.array(dados[500])

In [7]:
#normalizando os atributos para uma mesma escala
mm = MinMaxScaler()
X_norm = mm.fit_transform(X)

#separando os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2)

In [509]:
#treinando o algoritmo para k = 3
k = KNN(3)
k.fit(X_train, y_train)

#calculando a acurácia
k.score(X_test, y_test)

0.95

### Implementando o MDC

In [517]:
class MDC:
  """
  No MDC, cada classe de treino tem um centroide. Baseado nisso, recebemos novas amostras,
  calculamos seus centroides e classificamos de acordo com o centroide mais próximo entre
  os das classes de treino.
  """
  
  def __init__(self):
    pass

  def fit(self, X, y):
    self.X_train = X
    self.y_train = y

  def separate_classes(self, X, y):
    #separa as amostras por classes, para calcular o centroide de cada uma

    #encontra os valores únicos das classes
    m = list(set(y)) 
    #guarda os índices e os valores de y
    enum = list(enumerate(y)) 

    #dicionário com uma chave para cada classe
    dicionario_y = {int(i): [] for i in m} 

    for k, v in dicionario_y.items():
      for index, value in enum:
        #separa as classes das amostras por índices
        if int(value) == k: 
          v.append(index)

    #encontrando as linhas de X correspondentes às de y, por classe
    dicionario_x = {k: X[v] for k, v in dicionario_y.items()}

    return dicionario_x

  def calculate_centroids(self, X, y):
    #calcula os centroides de cada classe

    d = self.separate_classes(X, y)
    centroids = [[k, np.mean(v)] for k, v in d.items()]

    return centroids

  def predict(self, X_test):
    r = list()

    centroides_treino = self.calculate_centroids(self.X_train, self.y_train)

    #calculamos os centroides das amostras manualmente porque não temos os valores das classes, já que queremos prevê-las
    centroides_amostras = list(enumerate([np.mean(row) for row in X_test]))

    d = {i: [] for i in range(len(X_test))}

    for row in centroides_amostras:
      for i in range(len(centroides_treino)):
        #reservamos em uma lista: o indice da amostra atual, o índice do centroide da classe atual e o valor absoluto
        #da diferença entre o centroide da amostra atual e o centroide da classe atual
        a = [row[0], centroides_treino[i][0], abs(row[1] - centroides_treino[i][1])]
        r.append(a)

    r = np.array(r)

    respostas = list()

    for i in range(len(r) - 1):
      #verifica se estamos comparando as amostras de mesmo índice
      if r[i, 0] == r[i+1, 0]: 
        #verificamos, para cada amostra, o menor valor entre as diferenças de seu centroide e os centroides das classes de treino,
        #que corresponde ao terceiro elemento de r
        min_centroid = min(r[i], r[i+1], key = lambda x: x[2])
        respostas.append(min_centroid[1])

    return respostas
    

  def score(self, X_test, y_real):
    y_predito = self.predict(X_test)

    acertos = 0

    #calcula a proporção de quantas vezes o algoritmo fez uma previsão correta
    for i in range(len(y_real)):
      if y_real[i] == y_predito[i]:
        acertos += 1

    taxa_acerto = acertos / len(y_real)

    return taxa_acerto

## Questão 3 - Calcular a acurácia média após 10 realizações do MDC

In [520]:
#os dados já foram separados na proporção sugerida, para fazer os testes de implementação dos algoritmos

for i in range(10):
  acuracias_mdc = list()

  X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2)

  m = MDC()
  m.fit(X_train, y_train)
  acuracias_mdc.append(m.score(X_test, y_test))

print(f'Média de acurácia do MDC: {np.mean(acuracias_mdc)}')

Média de acurácia do MDC: 0.75


## Questão 4 - Implementação da k-fold cross-validation e teste do KNN com K variando de 2 até 10, para uma 5-fold cv

In [ ]:
def kfoldcv(X, y, k, objeto_knn):
  """
  O cross-validation utilizando k-fold consiste em dividir os dados em
  k partes iguais (ou aproximadamente iguais), para treinar com todos os dados
  
  Nesse método, escolhemos aleatoriamente uma parte do dataset para teste, e
  deixamos o restante para treino.
  """

  tamanho_fold = round(len(X)/k)

  #armazenamos e embaralhamos os índices dos dados para satisfazer a aleatoriedade
  indices = list(range(len(X)))
  random.shuffle(indices)

  #criando os índices dos intervalos de dados de acordo com o tamanho de cada fold
  folds = [indices[x:x+tamanho_fold] for x in range(0, len(indices), tamanho_fold)]

  acuracias = list()

  for i in range(k):
    indices_teste = folds[i]
    X_teste = X[indices_teste]
    y_teste = y[indices_teste]

    indices_treino = []
    for fold in folds:
      if fold != indices_teste:
        indices_treino.append(fold)

    #transformando a lista de listas em uma lista só
    indices_treino = [item for outer_list in indices_treino for item in outer_list]
    X_treino = X[indices_treino]
    y_treino = y[indices_treino]

    objeto_knn.fit(X_treino, y_treino)
    acuracias.append(objeto_knn.score(X_teste, y_teste))

  return np.mean(acuracias)

In [ ]:
for i in range(2, 11):
  k = KNN(i)

  print('Acurácia média para K = ' + str(i) + ': ' + str(kfoldcv(X_norm, y, 5, k)))

Acurácia média para K = 2: 0.97
Acurácia média para K = 3: 0.9400000000000001
Acurácia média para K = 4: 0.9800000000000001
Acurácia média para K = 5: 0.96
Acurácia média para K = 6: 0.93
Acurácia média para K = 7: 0.97
Acurácia média para K = 8: 0.99
Acurácia média para K = 9: 0.9200000000000002
Acurácia média para K = 10: 0.96


Para esse teste, o melhor valor de K foi 8.